In [1]:
import numpy as np
import lightgbm as lgb

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from structure.Dataset import Dataset
from structure.SklearnEnsemble import SklearnEnsemble
from structure.LGBEnsemble import LGBEnsemble
from predict import majority_voting

/Users/tomek/.virtualenvs/masters/lib/python3.6/site-packages/lightgbm/__init__.py:47: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [6]:
# Data
iris = load_iris()
train_data, val_data = Dataset(iris.data, iris.target, iris.feature_names, iris.target_names).split()

## Sklearn

In [3]:
tree = DecisionTreeClassifier(max_depth=2)

clf = AdaBoostClassifier(n_estimators=10, base_estimator=tree)
ensemble = SklearnEnsemble(clf)
ensemble.fit(train_data)

clf_preds = ensemble.clf.predict(val_data.X)
preds = ensemble.predict(val_data.X)

print(clf_preds, preds)
print(accuracy_score(val_data.y, preds))
print(accuracy_score(val_data.y, clf_preds))

[2 2 1 1 1 0 0 0 1 1 2 1 1 1 1] [2 2 2 2 2 0 0 0 2 2 2 2 2 2 2]
0.4666666666666667
0.9333333333333333


## Light GBM

In [7]:
clf = lgb.LGBMClassifier(n_estimators=10, objective='multiclass')
ensemble = LGBEnsemble(clf)
ensemble.fit(train_data)

clf_preds = ensemble.clf.predict(val_data.X)
preds = ensemble.predict(val_data.X)

print(clf_preds, preds)
print(accuracy_score(val_data.y, preds))
print(accuracy_score(val_data.y, clf_preds))

[0 0 2 2 2 0 1 1 2 1 2 2 0 0 0] [0 0 2 2 2 0 1 1 2 1 2 2 0 0 0]
0.8
0.8
